In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Race Results

In [4]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9902/832157024.py:10: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


In [5]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP=GP[GP["Position"]!=0]
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,4,,None,None,Williams,,0,Marc,Gené,Marc Gené,0,0,5,0,0,0 days 00:01:21.834000,NaT,NaT,NaT,,0,2003,Italian Grand Prix
1,14,,WEB,WEB,Jaguar,,0,Mark,Webber,Mark Webber,0,0,17,0,0,0 days 00:01:11.662000,NaT,NaT,NaT,,0,2003,Austrian Grand Prix
2,14,,WEB,WEB,Jaguar,,0,Mark,Webber,Mark Webber,0,0,14,0,0,0 days 00:01:13.269000,NaT,NaT,NaT,,0,2003,United States Grand Prix
3,14,,WEB,WEB,Jaguar,,0,Mark,Webber,Mark Webber,0,0,11,0,0,0 days 00:01:22.754000,NaT,NaT,NaT,,0,2003,Italian Grand Prix
4,14,,WEB,WEB,Jaguar,,0,Mark,Webber,Mark Webber,0,0,11,0,0,0 days 00:01:16.775000,NaT,NaT,NaT,,0,2003,German Grand Prix


In [6]:
#Get the counts by final position in race

In [10]:
GP_Race_Wins=GP[GP["Year"].isin(range(2021,2025))].pivot(index=['Year','GP'], columns=['Position'], values=["FullName"])
GP_Race_Wins.head()

FullName                                   \
Position                                 1               2                3    
Year GP                                                                        
2021 Abu Dhabi Grand Prix    Max Verstappen  Lewis Hamilton     Lando Norris   
     Austrian Grand Prix     Max Verstappen    Lando Norris     Sergio Perez   
     Azerbaijan Grand Prix  Charles Leclerc  Lewis Hamilton   Max Verstappen   
     Bahrain Grand Prix      Max Verstappen  Lewis Hamilton  Valtteri Bottas   
     Belgian Grand Prix      Max Verstappen  George Russell   Lewis Hamilton   

                                                                \
Position                                  4                 5    
Year GP                                                          
2021 Abu Dhabi Grand Prix       Sergio Perez      Carlos Sainz   
     Austrian Grand Prix      Lewis Hamilton   Valtteri Bottas   
     Azerbaijan Grand Prix      Pierre Gasly      Carlos Sainz   
     Bahrain Grand Prix      Charles Leclerc      Pierre Gasly   
     Belgian Grand Prix     Daniel Ricciardo  Sebastian Vettel   

                                                               \
Position                                  6                7    
Year GP                                                         
2021 Abu Dhabi Grand Prix    Valtteri Bottas  Charles Leclerc   
     Austrian Grand Prix        Pierre Gasly     Yuki Tsunoda   
     Azerbaijan Grand Prix      Lando Norris     Sergio Perez   
     Bahrain Grand Prix     Daniel Ricciardo     Lando Norris   
     Belgian Grand Prix         Pierre Gasly     Sergio Perez   

                                                               \
Position                                  8                9    
Year GP                                                         
2021 Abu Dhabi Grand Prix       Yuki Tsunoda     Esteban Ocon   
     Austrian Grand Prix    Sebastian Vettel   George Russell   
     Azerbaijan Grand Prix      Yuki Tsunoda  Fernando Alonso   
     Bahrain Grand Prix         Carlos Sainz  Fernando Alonso   
     Belgian Grand Prix      Valtteri Bottas     Esteban Ocon   

                                                                \
Position                                  10                11   
Year GP                                                          
2021 Abu Dhabi Grand Prix   Daniel Ricciardo   Fernando Alonso   
     Austrian Grand Prix        Lance Stroll      Carlos Sainz   
     Azerbaijan Grand Prix   Valtteri Bottas  Sebastian Vettel   
     Bahrain Grand Prix         Lance Stroll      Sergio Perez   
     Belgian Grand Prix         Lando Norris   Charles Leclerc   

                                                                  \
Position                                    12                13   
Year GP                                                            
2021 Abu Dhabi Grand Prix         Pierre Gasly      Lance Stroll   
     Austrian Grand Prix       Charles Leclerc  Daniel Ricciardo   
     Azerbaijan Grand Prix        Esteban Ocon  Daniel Ricciardo   
     Bahrain Grand Prix     Antonio Giovinazzi      Yuki Tsunoda   
     Belgian Grand Prix        Nicholas Latifi      Carlos Sainz   

                                                                    \
Position                                    14                  15   
Year GP                                                              
2021 Abu Dhabi Grand Prix   Antonio Giovinazzi    Sebastian Vettel   
     Austrian Grand Prix       Fernando Alonso  Antonio Giovinazzi   
     Azerbaijan Grand Prix      Kimi Räikkönen      George Russell   
     Bahrain Grand Prix         Kimi Räikkönen      George Russell   
     Belgian Grand Prix        Fernando Alonso        Lance Stroll   

                                                                 \
Position                                    16               17   
Year GP                                               

In [11]:
#Remove the column that says full name
GP_Race_Wins.columns = GP_Race_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins=GP_Race_Wins.reset_index()

GP_Race_Wins=GP_Race_Wins.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Race_Wins.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
66,2024,Australian Grand Prix,Max Verstappen,Carlos Sainz,Sergio Perez,Lando Norris,Charles Leclerc,Oscar Piastri,George Russell,Yuki Tsunoda,Lance Stroll,Fernando Alonso,Lewis Hamilton,Alexander Albon,Valtteri Bottas,Kevin Magnussen,Esteban Ocon,Nico Hulkenberg,Pierre Gasly,Daniel Ricciardo,Guanyu Zhou,NaN
67,2024,Austrian Grand Prix,Max Verstappen,Lando Norris,George Russell,Carlos Sainz,Lewis Hamilton,Charles Leclerc,Oscar Piastri,Sergio Perez,Nico Hulkenberg,Esteban Ocon,Daniel Ricciardo,Kevin Magnussen,Pierre Gasly,Yuki Tsunoda,Fernando Alonso,Alexander Albon,Lance Stroll,Valtteri Bottas,Logan Sargeant,Guanyu Zhou
68,2024,Bahrain Grand Prix,Max Verstappen,Charles Leclerc,George Russell,Carlos Sainz,Sergio Perez,Fernando Alonso,Lando Norris,Oscar Piastri,Lewis Hamilton,Nico Hulkenberg,Yuki Tsunoda,Lance Stroll,Alexander Albon,Daniel Ricciardo,Kevin Magnussen,Valtteri Bottas,Guanyu Zhou,Logan Sargeant,Esteban Ocon,Pierre Gasly
69,2024,British Grand Prix,George Russell,Lewis Hamilton,Lando Norris,Max Verstappen,Oscar Piastri,Nico Hulkenberg,Carlos Sainz,Lance Stroll,Alexander Albon,Fernando Alonso,Charles Leclerc,Logan Sargeant,Yuki Tsunoda,Guanyu Zhou,Daniel Ricciardo,Valtteri Bottas,Kevin Magnussen,Esteban Ocon,Sergio Perez,Pierre Gasly
70,2024,Canadian Grand Prix,George Russell,Max Verstappen,Lando Norris,Oscar Piastri,Daniel Ricciardo,Fernando Alonso,Lewis Hamilton,Yuki Tsunoda,Lance Stroll,Alexander Albon,Charles Leclerc,Carlos Sainz,Logan Sargeant,Kevin Magnussen,Pierre Gasly,Sergio Perez,Valtteri Bottas,Esteban Ocon,Nico Hulkenberg,Guanyu Zhou


In [14]:
import plotly.graph_objects as go

fig = go.Figure(go.Table
 (header={"values": GP_Race_Wins.columns},
  cells=dict(
    values=GP_Race_Wins.T.values,
    # fill_color=[GP_Race_Wins.Color],
    align='center', font=dict(color='white', size=12)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins.loc[GP_Race_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig.update_layout(
    title="Qualifying Results",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig.show()

In [ ]:
#Sprint Results

In [15]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_sprint`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq3 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9902/1496947003.py:10: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [16]:
GP_sprint=df_bq3
GP_sprint=GP_sprint.fillna(0)
GP_sprint["Position"]=GP_sprint["Position"].astype(int)
GP_sprint["GridPosition"]=GP_sprint["GridPosition"].astype(int)
GP_sprint["Points"]=GP_sprint["Points"].astype(int)
GP_sprint=GP_sprint[GP_sprint["Position"]!=0]
GP_sprint.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,22,Y TSUNODA,TSU,,RB,6692FF,,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,15,,0,0 days 00:01:28.687000,NaT,NaT,NaT,,0,2024,Miami Grand Prix
1,22,Y TSUNODA,TSU,,RB,6692FF,,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,19,,0,0 days 00:01:37.892000,NaT,NaT,NaT,,0,2024,Chinese Grand Prix
2,22,Y TSUNODA,TSU,,RB,6692FF,,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,14,,0,0 days 00:01:06.557000,0 days 00:01:05.960000,NaT,NaT,,0,2024,Austrian Grand Prix
3,3,D RICCIARDO,RIC,,RB,6692FF,,Daniel,Ricciardo,Daniel Ricciardo,https://media.formula1.com/d_driver_fallback_i...,AUS,4,,0,0 days 00:01:28.700000,0 days 00:01:28.122000,0 days 00:01:28.044000,NaT,,0,2024,Miami Grand Prix
4,3,D RICCIARDO,RIC,,RB,6692FF,,Daniel,Ricciardo,Daniel Ricciardo,https://media.formula1.com/d_driver_fallback_i...,AUS,14,,0,0 days 00:01:37.321000,0 days 00:01:36.553000,NaT,NaT,,0,2024,Chinese Grand Prix


In [17]:
#Get the counts by final position in race

In [18]:
GP_Sprint_Wins=GP_sprint[GP_sprint["Year"].isin(range(2021,2025))].pivot(index=['Year','GP'], columns=['Position'], values=["FullName"])
GP_Sprint_Wins.head()

FullName                                  \
Position                                 1               2               3    
Year GP                                                                       
2023 Austrian Grand Prix     Max Verstappen    Sergio Perez    Lando Norris   
     Azerbaijan Grand Prix  Charles Leclerc    Sergio Perez  Max Verstappen   
     Belgian Grand Prix      Max Verstappen   Oscar Piastri    Carlos Sainz   
     Qatar Grand Prix         Oscar Piastri    Lando Norris  Max Verstappen   
     São Paulo Grand Prix      Lando Norris  Max Verstappen    Sergio Perez   

                                                                              \
Position                                 4               5                6    
Year GP                                                                        
2023 Austrian Grand Prix    Nico Hulkenberg    Carlos Sainz  Charles Leclerc   
     Azerbaijan Grand Prix   George Russell    Carlos Sainz   Lewis Hamilton   
     Belgian Grand Prix     Charles Leclerc    Lando Norris     Pierre Gasly   
     Qatar Grand Prix        George Russell    Carlos Sainz  Charles Leclerc   
     São Paulo Grand Prix    George Russell  Lewis Hamilton     Yuki Tsunoda   

                                                               \
Position                                 7                 8    
Year GP                                                         
2023 Austrian Grand Prix    Fernando Alonso      Lance Stroll   
     Azerbaijan Grand Prix  Alexander Albon   Fernando Alonso   
     Belgian Grand Prix      Lewis Hamilton      Sergio Perez   
     Qatar Grand Prix       Nico Hulkenberg      Sergio Perez   
     São Paulo Grand Prix   Charles Leclerc  Daniel Ricciardo   

                                                              \
Position                                 9                10   
Year GP                                                        
2023 Austrian Grand Prix       Esteban Ocon  Kevin Magnussen   
     Azerbaijan Grand Prix     Lance Stroll     Lando Norris   
     Belgian Grand Prix        Esteban Ocon   George Russell   
     Qatar Grand Prix       Fernando Alonso     Esteban Ocon   
     São Paulo Grand Prix      Carlos Sainz    Oscar Piastri   

                                                               \
Position                                  11               12   
Year GP                                                         
2023 Austrian Grand Prix     Alexander Albon     Pierre Gasly   
     Azerbaijan Grand Prix     Oscar Piastri  Nico Hulkenberg   
     Belgian Grand Prix     Daniel Ricciardo  Fernando Alonso   
     Qatar Grand Prix           Pierre Gasly   Lewis Hamilton   
     São Paulo Grand Prix    Kevin Magnussen  Nico Hulkenberg   

                                                                               \
Position                                 13               14               15   
Year GP                                                                         
2023 Austrian Grand Prix       Yuki Tsunoda    Nyck De Vries   George Russell   
     Azerbaijan Grand Prix     Esteban Ocon  Kevin Magnussen   Logan Sargeant   
     Belgian Grand Prix     Alexander Albon   Logan Sargeant     Lance Stroll   
     Qatar Grand Prix       Valtteri Bottas      Liam Lawson      Guanyu Zhou   
     São Paulo Grand Prix      Pierre Gasly  Valtteri Bottas  Fernando Alonso   

                                                                            \
Position                              16               17               18   
Year GP                                                                      
2023 Austrian Grand Prix     Guanyu Zhou    Oscar Piastri   Lewis Hamilton   
     Azerbaijan Grand Prix   Guanyu Zhou  Valtteri Bottas     Yuki Tsunoda   
     Belgian Grand Prix     Yuki Tsunoda  Valtteri Bottas  Kevin Magnussen   
     Qatar Grand Prix       Lance Stroll  Alexander Albon     Yuki Tsunoda   
     São Pa

In [19]:
#Remove the column that says full name
GP_Sprint_Wins.columns = GP_Sprint_Wins.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins=GP_Sprint_Wins.reset_index()

GP_Sprint_Wins=GP_Sprint_Wins.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Sprint_Wins.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
6,2024,Austrian Grand Prix,Max Verstappen,Lando Norris,Oscar Piastri,George Russell,Carlos Sainz,Lewis Hamilton,Sergio Perez,Esteban Ocon,Pierre Gasly,Charles Leclerc,Kevin Magnussen,Lance Stroll,Fernando Alonso,Yuki Tsunoda,Logan Sargeant,Daniel Ricciardo,Nico Hulkenberg,Valtteri Bottas,Alexander Albon,Guanyu Zhou
7,2024,Chinese Grand Prix,Lando Norris,Lewis Hamilton,Fernando Alonso,Max Verstappen,Carlos Sainz,Sergio Perez,Charles Leclerc,Oscar Piastri,Valtteri Bottas,Guanyu Zhou,George Russell,Kevin Magnussen,Nico Hulkenberg,Daniel Ricciardo,Lance Stroll,Pierre Gasly,Esteban Ocon,Alexander Albon,Yuki Tsunoda,Logan Sargeant
8,2024,Miami Grand Prix,Max Verstappen,Charles Leclerc,Sergio Perez,Daniel Ricciardo,Carlos Sainz,Oscar Piastri,Lance Stroll,Fernando Alonso,Lando Norris,Nico Hulkenberg,George Russell,Lewis Hamilton,Esteban Ocon,Kevin Magnussen,Yuki Tsunoda,Pierre Gasly,Guanyu Zhou,Valtteri Bottas,Logan Sargeant,Alexander Albon
0,2023,Austrian Grand Prix,Max Verstappen,Sergio Perez,Lando Norris,Nico Hulkenberg,Carlos Sainz,Charles Leclerc,Fernando Alonso,Lance Stroll,Esteban Ocon,Kevin Magnussen,Alexander Albon,Pierre Gasly,Yuki Tsunoda,Nyck De Vries,George Russell,Guanyu Zhou,Oscar Piastri,Lewis Hamilton,Valtteri Bottas,Logan Sargeant
1,2023,Azerbaijan Grand Prix,Charles Leclerc,Sergio Perez,Max Verstappen,George Russell,Carlos Sainz,Lewis Hamilton,Alexander Albon,Fernando Alonso,Lance Stroll,Lando Norris,Oscar Piastri,Nico Hulkenberg,Esteban Ocon,Kevin Magnussen,Logan Sargeant,Guanyu Zhou,Valtteri Bottas,Yuki Tsunoda,Pierre Gasly,Nyck De Vries


In [20]:
import plotly.graph_objects as go

fig_sprint = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins.columns},
  cells=dict(
    values=GP_Sprint_Wins.T.values,
    # fill_color=[GP_Sprint_Wins.Color],
    align='center', font=dict(color='white', size=14)
    ))
 )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins.loc[GP_Sprint_Wins["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint.update_layout(
    title="Sprint Race Results",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig_sprint.show()

In [ ]:
#Team Race Results

In [ ]:
#Get the counts by final position in race

In [28]:
GP_Race_Wins_Team=GP[GP["Year"].isin(range(2021,2025))].pivot(index=['Year','GP'], columns=['Position'], values=["TeamName"])
GP_Race_Wins_Team.head()

TeamName                             \
Position                                 1         2                3    
Year GP                                                                  
2021 Abu Dhabi Grand Prix   Red Bull Racing  Mercedes          McLaren   
     Austrian Grand Prix    Red Bull Racing   McLaren  Red Bull Racing   
     Azerbaijan Grand Prix          Ferrari  Mercedes  Red Bull Racing   
     Bahrain Grand Prix     Red Bull Racing  Mercedes         Mercedes   
     Belgian Grand Prix     Red Bull Racing  Williams         Mercedes   

                                                                       \
Position                                 4             5           6    
Year GP                                                                 
2021 Abu Dhabi Grand Prix   Red Bull Racing       Ferrari    Mercedes   
     Austrian Grand Prix           Mercedes      Mercedes  AlphaTauri   
     Azerbaijan Grand Prix       AlphaTauri       Ferrari     McLaren   
     Bahrain Grand Prix             Ferrari    AlphaTauri     McLaren   
     Belgian Grand Prix             McLaren  Aston Martin  AlphaTauri   

                                                                     \
Position                                 7             8         9    
Year GP                                                               
2021 Abu Dhabi Grand Prix           Ferrari    AlphaTauri    Alpine   
     Austrian Grand Prix         AlphaTauri  Aston Martin  Williams   
     Azerbaijan Grand Prix  Red Bull Racing    AlphaTauri    Alpine   
     Bahrain Grand Prix             McLaren       Ferrari    Alpine   
     Belgian Grand Prix     Red Bull Racing      Mercedes    Alpine   

                                                                              \
Position                              10               11                 12   
Year GP                                                                        
2021 Abu Dhabi Grand Prix        McLaren           Alpine         AlphaTauri   
     Austrian Grand Prix    Aston Martin          Ferrari            Ferrari   
     Azerbaijan Grand Prix      Mercedes     Aston Martin             Alpine   
     Bahrain Grand Prix     Aston Martin  Red Bull Racing  Alfa Romeo Racing   
     Belgian Grand Prix          McLaren          Ferrari           Williams   

                                                             \
Position                              13                 14   
Year GP                                                       
2021 Abu Dhabi Grand Prix   Aston Martin  Alfa Romeo Racing   
     Austrian Grand Prix         McLaren             Alpine   
     Azerbaijan Grand Prix       McLaren  Alfa Romeo Racing   
     Bahrain Grand Prix       AlphaTauri  Alfa Romeo Racing   
     Belgian Grand Prix          Ferrari             Alpine   

                                                                  \
Position                                   15                 16   
Year GP                                                            
2021 Abu Dhabi Grand Prix        Aston Martin           Williams   
     Austrian Grand Prix    Alfa Romeo Racing  Alfa Romeo Racing   
     Azerbaijan Grand Prix           Williams           Williams   
     Bahrain Grand Prix              Williams             Alpine   
     Belgian Grand Prix          Aston Martin  Alfa Romeo Racing   

                                                             \
Position                              17                 18   
Year GP                                                       
2021 Abu Dhabi Grand Prix       Williams  Alfa Romeo Racing   
     Austrian Grand Prix          Alpine           Williams   
     Azerbaijan Grand Prix  Haas F1 Team       Haas F1 Team   
     Bahrain Grand Prix         Williams       Aston Martin   
     Belgian Grand Prix       AlphaTauri       Haas F1 Team   

                                                                  
Position                 

In [29]:
#Remove the column that says full name
GP_Race_Wins_Team.columns = GP_Race_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Race_Wins_Team=GP_Race_Wins_Team.reset_index()

GP_Race_Wins_Team=GP_Race_Wins_Team.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Race_Wins_Team.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
66,2024,Australian Grand Prix,Red Bull Racing,Ferrari,Red Bull Racing,McLaren,Ferrari,McLaren,Mercedes,RB,Aston Martin,Aston Martin,Mercedes,Williams,Kick Sauber,Haas F1 Team,Alpine,Haas F1 Team,Alpine,RB,Kick Sauber,NaN
67,2024,Austrian Grand Prix,Red Bull Racing,McLaren,Mercedes,Ferrari,Mercedes,Ferrari,McLaren,Red Bull Racing,Haas F1 Team,Alpine,RB,Haas F1 Team,Alpine,RB,Aston Martin,Williams,Aston Martin,Kick Sauber,Williams,Kick Sauber
68,2024,Bahrain Grand Prix,Red Bull Racing,Ferrari,Mercedes,Ferrari,Red Bull Racing,Aston Martin,McLaren,McLaren,Mercedes,Haas F1 Team,RB,Aston Martin,Williams,RB,Haas F1 Team,Kick Sauber,Kick Sauber,Williams,Alpine,Alpine
69,2024,British Grand Prix,Mercedes,Mercedes,McLaren,Red Bull Racing,McLaren,Haas F1 Team,Ferrari,Aston Martin,Williams,Aston Martin,Ferrari,Williams,RB,Kick Sauber,RB,Kick Sauber,Haas F1 Team,Alpine,Red Bull Racing,Alpine
70,2024,Canadian Grand Prix,Mercedes,Red Bull Racing,McLaren,McLaren,RB,Aston Martin,Mercedes,RB,Aston Martin,Williams,Ferrari,Ferrari,Williams,Haas F1 Team,Alpine,Red Bull Racing,Kick Sauber,Alpine,Haas F1 Team,Kick Sauber


In [30]:
import plotly.graph_objects as go

fig_team = go.Figure(go.Table
 (header={"values": GP_Race_Wins_Team.columns},
  cells=dict(
    values=GP_Race_Wins_Team.T.values,
    # fill_color=[GP_Race_Wins_Team.Color],
    align='center', font=dict(color='white', size=12)
    ))
 )

# fig_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Race_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Race_Wins_Team.loc[GP_Race_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Race_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_team.update_layout(
    title="Qualifying Results By Team",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig_team.show()

In [ ]:
#Team Sprint Race Results

In [31]:
GP_Sprint_Wins_Team=GP_sprint[GP_sprint["Year"].isin(range(2021,2025))].pivot(index=['Year','GP'], columns=['Position'], values=["TeamName"])
GP_Sprint_Wins_Team.head()

TeamName                                    \
Position                                 1                2                3    
Year GP                                                                         
2023 Austrian Grand Prix    Red Bull Racing  Red Bull Racing          McLaren   
     Azerbaijan Grand Prix          Ferrari  Red Bull Racing  Red Bull Racing   
     Belgian Grand Prix     Red Bull Racing          McLaren          Ferrari   
     Qatar Grand Prix               McLaren          McLaren  Red Bull Racing   
     São Paulo Grand Prix           McLaren  Red Bull Racing  Red Bull Racing   

                                                                              \
Position                              4         5           6             7    
Year GP                                                                        
2023 Austrian Grand Prix    Haas F1 Team   Ferrari     Ferrari  Aston Martin   
     Azerbaijan Grand Prix      Mercedes   Ferrari    Mercedes      Williams   
     Belgian Grand Prix          Ferrari   McLaren      Alpine      Mercedes   
     Qatar Grand Prix           Mercedes   Ferrari     Ferrari  Haas F1 Team   
     São Paulo Grand Prix       Mercedes  Mercedes  AlphaTauri       Ferrari   

                                                                         \
Position                                 8             9             10   
Year GP                                                                   
2023 Austrian Grand Prix       Aston Martin        Alpine  Haas F1 Team   
     Azerbaijan Grand Prix     Aston Martin  Aston Martin       McLaren   
     Belgian Grand Prix     Red Bull Racing        Alpine      Mercedes   
     Qatar Grand Prix       Red Bull Racing  Aston Martin        Alpine   
     São Paulo Grand Prix        AlphaTauri       Ferrari       McLaren   

                                                                    \
Position                              11            12          13   
Year GP                                                              
2023 Austrian Grand Prix        Williams        Alpine  AlphaTauri   
     Azerbaijan Grand Prix       McLaren  Haas F1 Team      Alpine   
     Belgian Grand Prix       AlphaTauri  Aston Martin    Williams   
     Qatar Grand Prix             Alpine      Mercedes  Alfa Romeo   
     São Paulo Grand Prix   Haas F1 Team  Haas F1 Team      Alpine   

                                                                      \
Position                              14            15            16   
Year GP                                                                
2023 Austrian Grand Prix      AlphaTauri      Mercedes    Alfa Romeo   
     Azerbaijan Grand Prix  Haas F1 Team      Williams    Alfa Romeo   
     Belgian Grand Prix         Williams  Aston Martin    AlphaTauri   
     Qatar Grand Prix         AlphaTauri    Alfa Romeo  Aston Martin   
     São Paulo Grand Prix     Alfa Romeo  Aston Martin        Alpine   

                                                                      \
Position                              17            18            19   
Year GP                                                                
2023 Austrian Grand Prix         McLaren      Mercedes    Alfa Romeo   
     Azerbaijan Grand Prix    Alfa Romeo    AlphaTauri        Alpine   
     Belgian Grand Prix       Alfa Romeo  Haas F1 Team    Alfa Romeo   
     Qatar Grand Prix           Williams    AlphaTauri  Haas F1 Team   
     São Paulo Grand Prix   Aston Martin    Alfa Romeo      Williams   

                                          
Position                              20  
Year GP                                   
2023 Austrian Grand Prix        Williams  
     Azerbaijan Grand Prix    AlphaTauri  
     Belgian Grand Prix     Haas F1 Team  
     Qatar Grand Prix           Williams  
     São Paulo Grand Prix       Williams

In [32]:
#Remove the column that says full name
GP_Sprint_Wins_Team.columns = GP_Sprint_Wins_Team.columns.droplevel(level=0)

#Reset the index to get a proper df
GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.reset_index()

GP_Sprint_Wins_Team=GP_Sprint_Wins_Team.sort_values(by=["Year","GP"],ascending=[False,True])
GP_Sprint_Wins_Team.head()

Position,Year,GP,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
6,2024,Austrian Grand Prix,Red Bull Racing,McLaren,McLaren,Mercedes,Ferrari,Mercedes,Red Bull Racing,Alpine,Alpine,Ferrari,Haas F1 Team,Aston Martin,Aston Martin,RB,Williams,RB,Haas F1 Team,Kick Sauber,Williams,Kick Sauber
7,2024,Chinese Grand Prix,McLaren,Mercedes,Aston Martin,Red Bull Racing,Ferrari,Red Bull Racing,Ferrari,McLaren,Kick Sauber,Kick Sauber,Mercedes,Haas F1 Team,Haas F1 Team,RB,Aston Martin,Alpine,Alpine,Williams,RB,Williams
8,2024,Miami Grand Prix,Red Bull Racing,Ferrari,Red Bull Racing,RB,Ferrari,McLaren,Aston Martin,Aston Martin,McLaren,Haas F1 Team,Mercedes,Mercedes,Alpine,Haas F1 Team,RB,Alpine,Kick Sauber,Kick Sauber,Williams,Williams
0,2023,Austrian Grand Prix,Red Bull Racing,Red Bull Racing,McLaren,Haas F1 Team,Ferrari,Ferrari,Aston Martin,Aston Martin,Alpine,Haas F1 Team,Williams,Alpine,AlphaTauri,AlphaTauri,Mercedes,Alfa Romeo,McLaren,Mercedes,Alfa Romeo,Williams
1,2023,Azerbaijan Grand Prix,Ferrari,Red Bull Racing,Red Bull Racing,Mercedes,Ferrari,Mercedes,Williams,Aston Martin,Aston Martin,McLaren,McLaren,Haas F1 Team,Alpine,Haas F1 Team,Williams,Alfa Romeo,Alfa Romeo,AlphaTauri,Alpine,AlphaTauri


In [33]:
import plotly.graph_objects as go

fig_sprint_team = go.Figure(go.Table
 (header={"values": GP_Sprint_Wins_Team.columns},
  cells=dict(
    values=GP_Sprint_Wins_Team.T.values,
    # fill_color=[GP_Sprint_Wins_Team.Color],
    align='center', font=dict(color='white', size=14)
    ))
 )

# fig_sprint_team.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": GP_Sprint_Wins_Team.T.values
#                                 if c == "All"
#                                 else GP_Sprint_Wins_Team.loc[GP_Sprint_Wins_Team["Year"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + GP_Sprint_Wins_Team["Year"].unique().tolist()
#             ]
#         }
#     ]
# )

fig_sprint_team.update_layout(
    title="Sprint Race Results By Team",
        title_x=0.5,
        template="plotly_dark",

    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="White"
    ),
    height=1250,
    width=2000,
    title_font_family="PT Sans Narrow"
)
fig_sprint_team.show()